In [ ]:
%pip install peft==0.4.0
%pip install datasets 
%pip install ipywidgets
!pip install transformers torch accelerate

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from huggingface_hub import login
login()

We will re-use the same dataset and model from the demo notebook. 

In [4]:
!huggingface-cli whoami

Ayush-1722


In [8]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data["train"].select(range(50))
display(train_sample) 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [9]:
import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    lora_dropout=0.05, 
    bias="none", # this specifies if the bias parameter should be trained. 
    task_type="CAUSAL_LM"
)

In [10]:
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 524,288 || all params: 6,738,939,904 || trainable%: 0.007779977377284533
None


In [11]:
%%time 

import transformers
from transformers import TrainingArguments, Trainer
import os

output_directory = os.path.join("models", "LoRA_outputs_Llama_7B")

if not os.path.exists("models"):
    os.mkdir("models")
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
    
training_args = TrainingArguments(
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=5,
    no_cuda=True
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


CPU times: user 13h 40min 47s, sys: 8min 46s, total: 13h 49min 34s
Wall time: 50min 5s


TrainOutput(global_step=35, training_loss=7.988033621651786, metrics={'train_runtime': 3005.6278, 'train_samples_per_second': 0.083, 'train_steps_per_second': 0.012, 'total_flos': 1481616141434880.0, 'train_loss': 7.988033621651786, 'epoch': 5.0})

In [12]:
import time

time_now = time.time()

peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")

trainer.model.save_pretrained(peft_model_path)

In [13]:
from peft import PeftModel, PeftConfig

loaded_model = PeftModel.from_pretrained(foundation_model, 
                                        peft_model_path, 
                                        is_trainable=False)

## Inference

In [14]:
inputs = tokenizer("Two things are infinite: ", return_tensors="pt")
outputs = peft_model.generate(
    input_ids=inputs["input_ids"], 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=7, 
    eos_token_id=tokenizer.eos_token_id
    )
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Two things are infinite:  you The “ is the to who']
